In [2]:
data = "/content/drive/MyDrive/Social Computing/database.sqlite"
print("Using DB:", data)

Using DB: /content/drive/MyDrive/Social Computing/database.sqlite


In [3]:
import sqlite3, pandas as pd

pd.set_option("display.max_colwidth", 120)
conn = sqlite3.connect(data)

# Helper to run SQL and return a DataFrame
def q(sql, params=None):
    return pd.read_sql(sql, conn, params=params or {})

# Exercise 1.1: Reading the dataset

In [ ]:
tables = q("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
tables

,name
0,comments
1,follows
2,posts
3,reactions
4,sqlite_sequence
5,users


In [ ]:
from typing import Dict, Any

def example_values(table: str, n: int = 3) -> Dict[str, Any]:
    df = q(f"SELECT * FROM {table} LIMIT 200")
    out = {}
    for col in df.columns:

        ex = df[col].dropna().astype(str)
        out[col] = ex.iloc[0] if len(ex) else None
    return out

rows = []
for t in tables['name']:
    if t == 'sqlite_sequence':
        continue
    schema = q(f"PRAGMA table_info({t});")
    count = q(f"SELECT COUNT(*) AS n FROM {t};")['n'][0]
    examples = example_values(t)

    # expand one row per column for a descriptive table
    for _, r in schema.iterrows():
        rows.append({
            "table": t,
            "row_count": count,
            "column": r["name"],
            "type": r["type"],
            "not_null": bool(r["notnull"]),
            "default": r["dflt_value"],
            "pk": bool(r["pk"]),
            "example_value": examples.get(r["name"])
        })

schema_df = pd.DataFrame(rows)
schema_df.sort_values(["table", "pk"], ascending=[True, False]).reset_index(drop=True)

,table,row_count,column,type,not_null,default,pk,example_value
0,comments,5804,id,INTEGER,False,None,True,1
1,comments,5804,post_id,INTEGER,True,None,False,1963
2,comments,5804,user_id,INTEGER,True,None,False,55
3,comments,5804,content,TEXT,True,None,False,"Haha, I bet your neighbors are either loving or hating you right now! Crank it up and see if you can get a dance par..."
4,comments,5804,created_at,TIMESTAMP,True,CURRENT_TIMESTAMP,False,2022-12-04 02:36:15
5,follows,7225,follower_id,INT,True,None,False,12
6,follows,7225,followed_id,INT,True,None,False,1
7,posts,1303,id,INTEGER,False,None,True,1718
8,posts,1303,user_id,INTEGER,True,None,False,10
9,posts,1303,content,TEXT,True,None,False,Just had the most ridiculous encounter with a cat in Shibuya. It hissed like I was invading its turf! #CatWhisperer ...


# Exercise 1.2: Lurkers

In [ ]:
lurkers_sql = """
SELECT COUNT(*) AS lurkers_count
FROM users u
WHERE u.id NOT IN (SELECT DISTINCT user_id FROM posts)
  AND u.id NOT IN (SELECT DISTINCT user_id FROM comments)
  AND u.id NOT IN (SELECT DISTINCT user_id FROM reactions);
"""
lurkers_count = q(lurkers_sql)['lurkers_count'][0]
lurkers_count


np.int64(55)

In [ ]:
lurkers_list_sql = """
SELECT u.id, u.username
FROM users u
WHERE u.id NOT IN (SELECT DISTINCT user_id FROM posts)
  AND u.id NOT IN (SELECT DISTINCT user_id FROM comments)
  AND u.id NOT IN (SELECT DISTINCT user_id FROM reactions)
ORDER BY u.id;
"""
lurkers_df = q(lurkers_list_sql)
lurkers_df.head(20)

,id,username
0,474,starry_night
1,475,breezyday22
2,476,wanderlust89
3,477,tech_savvy
4,478,coffee_lover
5,479,bookworm77
6,480,traveler_guy
7,481,sunshine_day
8,482,nightowl22
9,483,music_lover


# Exercise 1.3: Influencers

## I think here Engagement means (comments + reactions) received on a user’s posts.

In [6]:
influencers_sql = """
WITH post_eng AS (
  SELECT p.user_id,
         COUNT(DISTINCT c.id) AS comment_count,
         COUNT(DISTINCT r.id) AS reaction_count,
         (COUNT(DISTINCT c.id) + COUNT(DISTINCT r.id)) AS total_engagement
  FROM posts p
  LEFT JOIN comments c ON c.post_id = p.id
  LEFT JOIN reactions r ON r.post_id = p.id
  GROUP BY p.user_id
)
SELECT u.id AS user_id, u.username, comment_count, reaction_count, total_engagement
FROM post_eng pe
JOIN users u ON u.id = pe.user_id
ORDER BY total_engagement DESC
LIMIT 5;
"""
top5_influencers = q(influencers_sql)
top5_influencers


,user_id,username,comment_count,reaction_count,total_engagement
0,54,WinterWolf,179,267,446
1,65,PinkPanther,152,234,386
2,94,PinkPetal,137,246,383
3,81,GoldenDreams,149,217,366
4,30,WildHorse,157,196,353


# Exercise 1.4: Spammers

In [7]:
spammers_detail_sql = """
WITH all_texts AS (
  SELECT user_id, content FROM posts
  UNION ALL
  SELECT user_id, content FROM comments
),
repeats AS (
  SELECT user_id, content, COUNT(*) AS times_shared
  FROM all_texts
  GROUP BY user_id, content
  HAVING COUNT(*) >= 3
)
SELECT u.id AS user_id, u.username, r.content, r.times_shared
FROM repeats r
JOIN users u ON u.id = r.user_id
ORDER BY r.times_shared DESC, u.id ASC;
"""
spammers_detail = q(spammers_detail_sql)
spammers_detail.head(20)


,user_id,username,content,times_shared
0,513,night_owl,? Mega Giveaway Alert! ? Follow all accounts we’re following and DM us to enter! ? #Freebie,8
1,530,coding_whiz,Top 10 gadgets of 2025 – All available here: bestbuystuff.click/top10,8
2,513,night_owl,?FLASH GIVEAWAY? Click the link in our bio to claim your PS5! Only 100 units left! #giveaway #win,7
3,533,eco_warrior,"Not gonna lie, I was skeptical at first. But after tagging three friends and waiting 24 hours, I actually got my fre...",7
4,521,history_buff,A lot of you asked what helped me drop 5kg in a month—this tea really works! Here’s my referral link: fittearesults[...,5
5,521,history_buff,Best way to clean your sneakers ? snag yours here ➡️ whitelabelshop.biz/sneakerclean,5
6,521,history_buff,Mood: me refreshing for likes every 30 seconds #relatable #socialmediaproblems #algorithmwarrior #realonesknow #lmaooo,5
7,524,yoga_yogi,"I couldn’t believe it! I just entered this giveaway and actually won a brand-new iPhone 15. Thought it was fake, but...",5
8,530,coding_whiz,Shocking! #lol #weekend #coffee #bookstagram #design #artist #blogger #aesthetic,5
9,513,night_owl,?FREE VACATION? Tag a friend you’d take to Bali! Winners picked tonight! #TravelGiveaway,4


In [ ]:
spammers_summary_sql = """
WITH all_texts AS (
  SELECT user_id, content FROM posts
  UNION ALL
  SELECT user_id, content FROM comments
),
repeats AS (
  SELECT user_id, content, COUNT(*) AS times_shared
  FROM all_texts
  GROUP BY user_id, content
  HAVING COUNT(*) >= 3
)
SELECT u.id AS user_id, u.username,
       COUNT(*) AS distinct_repeated_texts,
       SUM(times_shared) AS total_repeated_instances
FROM repeats r
JOIN users u ON u.id = r.user_id
GROUP BY u.id, u.username
ORDER BY total_repeated_instances DESC, distinct_repeated_texts DESC;
"""
spammers_summary = q(spammers_summary_sql)
spammers_summary


,user_id,username,distinct_repeated_texts,total_repeated_instances
0,513,night_owl,5,26
1,521,history_buff,5,22
2,530,coding_whiz,3,16
3,533,eco_warrior,3,14
4,524,yoga_yogi,2,8
